# Exercise: FPGA and the DevCloud

The following are the tasks assigned to you for this exercise, 

1. Load the license plate detection model on to the Intel Arria 10 FPGA and perform an inference on it.
2. Calculate the time it takes to do the Inference for 10 iterations

<span class="graffiti-highlight graffiti-id_vskulnq-id_oudamc9"><i></i><button>Introduction</button></span>



#### Set up paths so we can run Dev Cloud utilities
You *must* run this every time they enter a Workspace session.

In [2]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))

env: PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support


In [10]:
!find /opt/intel -name '*.sh'

/opt/intel/mediasdk/install_media.sh
/opt/intel/openvino_2020.1.023/openvino_toolkit_uninstaller/uninstall.sh
/opt/intel/openvino_2020.1.023/openvino_toolkit_uninstaller/uninstall_GUI.sh
/opt/intel/openvino_2020.1.023/deployment_tools/tools/cl_compiler/bin/cltools_setenv.sh
/opt/intel/openvino_2020.1.023/deployment_tools/tools/workbench/wait_until_db_is_live.sh
/opt/intel/openvino_2020.1.023/deployment_tools/tools/workbench/run_openvino_workbench.sh
/opt/intel/openvino_2020.1.023/deployment_tools/tools/workbench/docker-entrypoint.sh
/opt/intel/openvino_2020.1.023/deployment_tools/tools/workbench/start_rabbitmq.sh
/opt/intel/openvino_2020.1.023/deployment_tools/tools/workbench/build_docker.sh
/opt/intel/openvino_2020.1.023/deployment_tools/demo/demo_squeezenet_download_convert_run.sh
/opt/intel/openvino_2020.1.023/deployment_tools/demo/utils.sh
/opt/intel/openvino_2020.1.023/deployment_tools/demo/demo_security_barrier_camera.sh
/opt/intel/openvino_2020.1.023/deployment_tools/demo/demo_b

## The model

We will be using the `vehicle-license-plate-detection-barrier-0106` model for this exercise. Remember that to run a model on the FPGA, we need to use FP16 as the model precision.

The model is present in the `/data/models/intel` folder.

# Step 1: Creating a Python Script

The first step is to create a python script that you can use to load the model and perform an inference. I have used the `writefile` magic to create a python file called `inference_on_device.py`. You will need to complete this file.

In [11]:
%%writefile inference_on_device.py

import time
import cv2
import numpy as np
from openvino.inference_engine import IENetwork
from openvino.inference_engine import IECore
import argparse
import os.path

def main(args):
    model=args.model_path
    model_weights=model+'.bin'
    model_structure=model+'.xml'
    
    print('#', model_weights, 'Exists:', os.path.isfile(model_weights))
    print('#', model_structure, 'Exists:', os.path.isfile(model_structure))
    
    start=time.time()
    model=IENetwork(model_structure, model_weights)

    core = IECore()
    net = core.load_network(network=model, device_name=args.device, num_requests=1)
    print(f"Time taken to load model = {time.time()-start} seconds")

    # Get the name of the input node
    input_name=next(iter(model.inputs))

    # Reading and Preprocessing Image
    input_img=cv2.imread('/data/resources/car.png')
    input_img=cv2.resize(input_img, (300,300), interpolation = cv2.INTER_AREA)
    input_img=np.moveaxis(input_img, -1, 0)

    # Running Inference in a loop on the same image
    input_dict={input_name:input_img}

    start=time.time()
    for _ in range(10):
        net.infer(input_dict)
    
    print(f"Time Taken to run 10 Inference on FPGA is = {time.time()-start} seconds")

if __name__=='__main__':
    parser=argparse.ArgumentParser()
    parser.add_argument('--model_path', required=True)
    parser.add_argument('--device', default=None)
    
    args=parser.parse_args() 
    main(args)

Overwriting inference_on_device.py


<span class="graffiti-highlight graffiti-id_f28ff2h-id_4psdryf"><i></i><button>Show Solution</button></span>

## Step 2: Creating a job submission script

To submit a job to the devcloud, we need to create a script. I have named the script as `inference_fpga_model_job.sh`.

Can you write a script that will take the model path and device as a command line argument and then call the python file you created in the previous cell with the path to the model?

Remember that you will need to use the `aocl` command to load the bitstream file on to the FPGA

In [12]:
%%writefile inference_fpga_model_job.sh
#!/bin/bash

exec 1>/output/stdout.log 2>/output/stderr.log

mkdir -p /output

DEVICE=$1
MODELPATH=$2

echo '---'
ls /opt/intel/
echo '---'

source /opt/intel/openvino/bin/setupvars.sh
source /opt/intel/init_openvino.sh
aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg1_bitstreams/2019R4_PL1_FP16_MobileNet_Clamp.aocx


# Run the load model python script
python3 inference_on_device.py  --model_path ${MODELPATH} --device ${DEVICE}

cd /output

tar zcvf output.tgz stdout.log stderr.log

Overwriting inference_fpga_model_job.sh


<span class="graffiti-highlight graffiti-id_5e0vxvt-id_5zk2mzh"><i></i><button>Show Solution</button></span>

## Step 3: Running on the FPGA

In the cell below, can you write the qsub command that will submit your job to the FPGA?

In [13]:
job_id_core = !qsub inference_fpga_model_job.sh -d . -l nodes=1:tank-870:i5-6500te:iei-mustang-f100-a10 -F "HETERO:FPGA,CPU /data/models/intel/vehicle-license-plate-detection-barrier-0106/FP32/vehicle-license-plate-detection-barrier-0106" -N store_core 
print(job_id_core[0])

KnQzQ6U1I7I5FVEWqupW20dZXOtkTZoB


<span class="graffiti-highlight graffiti-id_yr40vov-id_cvo0xg6"><i></i><button>Show Solution</button></span>

## Step 4: Getting the Live Stat Values

By running the below command, we can see the live status of the commands.

<span class="graffiti-highlight graffiti-id_ecvm8yr-id_nnpaoep"><i></i><button>Click here to know about liveqstat command</button></span>

In [14]:
import liveQStat
liveQStat.liveQStat()

## Step 5: Get the results

Running the cell below will get the output files from our job

<span class="graffiti-highlight graffiti-id_s7wimuv-id_xm8qs9p"><i></i><button> Click here to know about fetching the output files containing the results</button></span>

In [15]:
import get_results

get_results.getResults(job_id_core[0], filename="output.tgz", blocking=True)

getResults() is blocking until results of the job (id:KnQzQ6U1I7I5FVEWqupW20dZXOtkTZoB) are ready.
Please wait................Success!
output.tgz was downloaded in the same folder as this notebook.


In [16]:
!tar zxf output.tgz

In [17]:
!cat stdout.log

---
intel_sdp_products.db
mediasdk
openvino
openvino_2020.1.023
---
[setupvars.sh] OpenVINO environment initialized
# /data/models/intel/vehicle-license-plate-detection-barrier-0106/FP32/vehicle-license-plate-detection-barrier-0106.bin Exists: True
# /data/models/intel/vehicle-license-plate-detection-barrier-0106/FP32/vehicle-license-plate-detection-barrier-0106.xml Exists: True


In [18]:
!cat stderr.log

./submission.sh: line 15: /opt/intel/init_openvino.sh: No such file or directory
./submission.sh: line 16: aocl: command not found
Traceback (most recent call last):
  File "inference_on_device.py", line 48, in <module>
    main(args)
  File "inference_on_device.py", line 22, in main
    net = core.load_network(network=model, device_name=args.device, num_requests=1)
  File "ie_api.pyx", line 134, in openvino.inference_engine.ie_api.IECore.load_network
  File "ie_api.pyx", line 141, in openvino.inference_engine.ie_api.IECore.load_network
RuntimeError: Failed to create plugin libdliaPlugin.so for device FPGA
Please, check your environment
Cannot load library 'libdliaPlugin.so': libdliaPlugin.so: cannot open shared object file: No such file or directory

tar: stdout.log: file changed as we read it
